<a href="https://colab.research.google.com/github/chrismarkella/Kaggle-access-from-Google-Colab/blob/master/sql_vs_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np
import pandas as pd

from getpass import getpass 

In [2]:
def access_kaggle():
    """
    Access Kaggle from Google Colab.
    If the /root/.kaggle does not exist then prompt for
    the username and for the Kaggle API key.
    Creates the kaggle.json access file in the /root/.kaggle/ folder. 
    """
    KAGGLE_ROOT = os.path.join('/root', '.kaggle')
    KAGGLE_PATH = os.path.join(KAGGLE_ROOT, 'kaggle.json')

    if '.kaggle' not in os.listdir(path='/root'):
        user = getpass(prompt='Kaggle username: ')
        key  = getpass(prompt='Kaggle API key: ')
        
        !mkdir $KAGGLE_ROOT
        !touch $KAGGLE_PATH
        !chmod 666 $KAGGLE_PATH
        with open(KAGGLE_PATH, mode='w') as f:
            f.write('{"username":"%s", "key":"%s"}' %(user, key))
            f.close()
        !chmod 600 $KAGGLE_PATH
        del user
        del key
        success_msg = "Kaggle is successfully set up. Good to go."
        print(f'{success_msg}')

access_kaggle()


Kaggle username: ··········
Kaggle API key: ··········
Kaggle is successfully set up. Good to go.


In [3]:
!kaggle datasets download fernandol/countries-of-the-world --unzip
!mv countries\ of\ the\ world.csv countries_of_the_world.csv

df = pd.read_csv('countries_of_the_world.csv', sep=',')
df.head()

  0% 0.00/13.4k [00:00<?, ?B/s]
100% 13.4k/13.4k [00:00<00:00, 11.9MB/s]


,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


In [4]:
df.shape

(227, 20)

In [5]:
df.columns

Index(['Country', 'Region', 'Population', 'Area (sq. mi.)',
       'Pop. Density (per sq. mi.)', 'Coastline (coast/area ratio)',
       'Net migration', 'Infant mortality (per 1000 births)',
       'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)',
       'Crops (%)', 'Other (%)', 'Climate', 'Birthrate', 'Deathrate',
       'Agriculture', 'Industry', 'Service'],
      dtype='object')

In [7]:
df.columns = df.columns.str.strip().str.lower()
df.columns

Index(['country', 'region', 'population', 'area (sq. mi.)',
       'pop. density (per sq. mi.)', 'coastline (coast/area ratio)',
       'net migration', 'infant mortality (per 1000 births)',
       'gdp ($ per capita)', 'literacy (%)', 'phones (per 1000)', 'arable (%)',
       'crops (%)', 'other (%)', 'climate', 'birthrate', 'deathrate',
       'agriculture', 'industry', 'service'],
      dtype='object')

In [9]:
column_name_dict = {
    'area (sq. mi.)':'area',
    'pop. density (per sq. mi.)':'pop_density',
    'coastline (coast/area ratio)':'coastline',
    'infant mortality (per 1000 births)':'infant_mortality',
    'gdp ($ per capita)':'gdp_per_capita',
    'literacy (%)':'literacy',
    'phones (per 1000)':'phones',
    'arable (%)':'arable',
    'crops (%)':'crops',
    'other (%)':'other',
}

df.rename(columns=column_name_dict, inplace=True)
df.columns

Index(['country', 'region', 'population', 'area', 'pop_density', 'coastline',
       'net migration', 'infant_mortality', 'gdp_per_capita', 'literacy',
       'phones', 'arable', 'crops', 'other', 'climate', 'birthrate',
       'deathrate', 'agriculture', 'industry', 'service'],
      dtype='object')

In [11]:
df.columns = df.columns.map(lambda c: '_'.join(c.split()))
df.columns

Index(['country', 'region', 'population', 'area', 'pop_density', 'coastline',
       'net_migration', 'infant_mortality', 'gdp_per_capita', 'literacy',
       'phones', 'arable', 'crops', 'other', 'climate', 'birthrate',
       'deathrate', 'agriculture', 'industry', 'service'],
      dtype='object')

##SELECT


###SELECT 1
```SQL
SELECT population FROM world
  WHERE name = 'Germany'
```



In [12]:
filt_germany = (df.country == 'Germany')
df.loc[filt_germany, 'population']

Series([], Name: population, dtype: int64)

In [14]:
df.country.unique()[:10]

array(['Afghanistan ', 'Albania ', 'Algeria ', 'American Samoa ',
       'Andorra ', 'Angola ', 'Anguilla ', 'Antigua & Barbuda ',
       'Argentina ', 'Armenia '], dtype=object)

In [16]:
df.country.str.strip().unique()[:10]

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua & Barbuda', 'Argentina', 'Armenia'],
      dtype=object)

In [17]:
df.country = df.country.str.strip()
df.country.unique()[:5]

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra'],
      dtype=object)

In [18]:
filt_germany = (df.country == 'Germany')
df.loc[filt_germany, 'population']

76    82422299
Name: population, dtype: int64

###SELECT 2


```SQL
SELECT name, population FROM world
  WHERE name IN ('Brazil', 'Russia', 'India', 'China');
```



In [19]:
selected_countries = [
    'Brazil',
    'Russia',
    'India',
    'China',
]
filt_country = (df.country.isin(selected_countries))

df.loc[filt_country, ['country', 'population']]

,country,population
27,Brazil,188078227
42,China,1313973713
94,India,1095351995
169,Russia,142893540


###SELECT 3


```SQL
SELECT name, area FROM world
  WHERE area BETWEEN 250000 AND 300000
```



In [20]:
filt_area = df.area.between(250*10**3, 300*10**3)

selected_columns = [
    'country',
    'area',
]

df.loc[filt_area, selected_columns]

,country,area
31,Burkina Faso,274200
59,Ecuador,283560
72,Gabon,267667
149,New Zealand,268680
162,Philippines,300000
223,Western Sahara,266000


###SELECT 4


```SQL
SELECT name FROM world
  WHERE name LIKE 'F%'
```



In [21]:
filt_country_f = (df.country.map(lambda c: c[0] == 'F'))

df.loc[filt_country_f, 'country']

66       Faroe Islands
67                Fiji
68             Finland
69              France
70       French Guiana
71    French Polynesia
Name: country, dtype: object

###SELECT 5


```SQL
SELECT name FROM world
  WHERE name LIKE '%Y'
```



In [22]:
filt_country_end_y = (df.country.map(lambda c: c[-1] == 'y'))

df.loc[filt_country_end_y, 'country']

76      Germany
85     Guernsey
92      Hungary
101       Italy
104      Jersey
154      Norway
160    Paraguay
206      Turkey
215     Uruguay
Name: country, dtype: object

###SELECT 6


```SQL
SELECT name FROM world
  WHERE name LIKE '%x%'
```



In [23]:
filt_country_contains = (df.country.map(lambda c: 'x' in c))

df.loc[filt_country_contains, 'country']

121    Luxembourg
135        Mexico
Name: country, dtype: object

###SELECT 7



```SQL
SELECT name FROM world
  WHERE name LIKE '%land'
```



In [24]:
filt_land = (df.country.map(lambda c: c[-4:] == 'land'))

df.loc[filt_land, 'country']

68         Finland
80       Greenland
93         Iceland
98         Ireland
149    New Zealand
163         Poland
194      Swaziland
196    Switzerland
201       Thailand
Name: country, dtype: object

###SELECT 8


```SQL
SELECT name FROM world
  WHERE name LIKE 'C%ia'
```



In [25]:
filt_C_ia = (df.country.map(lambda c: c[0]=='C' and c[-2:]=='ia'))

df.loc[filt_C_ia, 'country']

34    Cambodia
43    Colombia
50     Croatia
Name: country, dtype: object

###SELECT 9


```SQL
SELECT name FROM world
  WHERE name LIKE '%oo%'
```



In [26]:
filt_oo = df.country.str.contains('oo')

df.loc[filt_oo, 'country']

35        Cameroon
47    Cook Islands
Name: country, dtype: object

In [30]:
filt_three_a = df.country.str.contains('.*a.*a.*a.*')

df.loc[filt_three_a, 'country']

3                        American Samoa
7                     Antigua & Barbuda
14                         Bahamas, The
36                               Canada
38                       Cayman Islands
62                    Equatorial Guinea
84                            Guatemala
102                             Jamaica
106                          Kazakhstan
124                          Madagascar
126                            Malaysia
130                    Marshall Islands
132                          Mauritania
150                           Nicaragua
153                  N. Mariana Islands
158                              Panama
159                    Papua New Guinea
160                            Paraguay
175    Saint Vincent and the Grenadines
179                        Saudi Arabia
200                            Tanzania
221                   Wallis and Futuna
223                      Western Sahara
Name: country, dtype: object

###SELECT 10


```SQL
SELECT name FROM world
 WHERE name LIKE '_t%'
```



In [32]:
filt_second_t = (df.country.str.contains('^.t.*'))

df.loc[filt_second_t, 'country']

65                 Ethiopia
101                   Italy
174    St Pierre & Miquelon
Name: country, dtype: object

###SELECT 11


```SQL
SELECT name FROM world
 WHERE name LIKE '%o__o%'
```



In [33]:
filt_o__o = (df.country.str.contains('.*o..o.*'))

df.loc[filt_o__o, 'country']

45         Congo, Dem. Rep.
46     Congo, Repub. of the
116                 Lesotho
137                 Moldova
139                Mongolia
141                 Morocco
178     Sao Tome & Principe
Name: country, dtype: object

###SELECT 12


```
SELECT name FROM world
 WHERE LEN(name) = 4
```



In [34]:
filt_len = (df.country.str.len() == 4)

df.loc[filt_len, 'country']

40     Chad
51     Cuba
67     Fiji
83     Guam
96     Iran
97     Iraq
113    Laos
128    Mali
155    Oman
161    Peru
202    Togo
Name: country, dtype: object

In [2]:
!ls -lh

total 72K
-rw-r--r-- 1 root root  67K Feb  2 07:49 nobel-laureates.zip
drwxr-xr-x 1 root root 4.0K Jan 13 16:38 sample_data


In [3]:
!unzip nobel-laureates.zip
!ls -lh

Archive:  nobel-laureates.zip
  inflating: archive.csv             
total 356K
-rw-r--r-- 1 root root 284K Sep 20 08:34 archive.csv
-rw-r--r-- 1 root root  67K Feb  2 07:49 nobel-laureates.zip
drwxr-xr-x 1 root root 4.0K Jan 13 16:38 sample_data


In [5]:
df = pd.read_csv('archive.csv', sep=',')
df.head(3)

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany


In [6]:
df.columns

Index(['Year', 'Category', 'Prize', 'Motivation', 'Prize Share', 'Laureate ID',
       'Laureate Type', 'Full Name', 'Birth Date', 'Birth City',
       'Birth Country', 'Sex', 'Organization Name', 'Organization City',
       'Organization Country', 'Death Date', 'Death City', 'Death Country'],
      dtype='object')

In [8]:
df.columns = df.columns.map(lambda c: '_'.join(c.lower().split()))
df.columns

Index(['year', 'category', 'prize', 'motivation', 'prize_share', 'laureate_id',
       'laureate_type', 'full_name', 'birth_date', 'birth_city',
       'birth_country', 'sex', 'organization_name', 'organization_city',
       'organization_country', 'death_date', 'death_city', 'death_country'],
      dtype='object')

###SELECT 13


```SQL
SELECT yr, subject, winner
  FROM nobel
 WHERE yr = 1960
```



In [9]:
filt_year = (df.year == 1960)
selected_columns = [
    'year',
    'category',
    'full_name',
]

df.loc[filt_year, selected_columns]

,year,category,full_name
338,1960,Chemistry,Willard Frank Libby
339,1960,Literature,Saint-John Perse
340,1960,Medicine,Sir Frank Macfarlane Burnet
341,1960,Medicine,Peter Brian Medawar
342,1960,Peace,Albert John Lutuli
343,1960,Physics,Donald Arthur Glaser
